<a href="https://colab.research.google.com/github/bala-221/ocrPDFToText/blob/main/scanToSearchPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Install and import packages**

In [16]:
import os
!pip install fpdf2
from fpdf import FPDF
import time
from PIL import Image
#!pip install pypandoc
#!sudo apt install texlive
#import pypandoc
!pip install PyMuPDF==1.19.0
!pip install install fitz
import fitz
!sudo apt install tesseract-ocr
!pip install pytesseract
import pytesseract
from pytesseract import Output, TesseractError
!pip install python-docx
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Mm
from google.colab import files


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**2. Upload the pdf file**

In [17]:

uploaded = files.upload()

Saving Abubakar Imam Memoirs-1-26.pdf to Abubakar Imam Memoirs-1-26.pdf


In [18]:
file = next(iter(uploaded)) #getting the file name uploaded

#check whether the file is an image or pdf

def check_image_with_pil(file):
 try:
       Image.open(file)
 except IOError:
        return False
 return True

if check_image_with_pil(file) == False: # it is a PDF
  iterable = fitz.open(file) #break the pdf file into pages
  image = False
else: # it is an image  
  iterable = range(0)




DEBUG:PIL.Image:Importing BlpImagePlugin
DEBUG:PIL.Image:Image: failed to import BlpImagePlugin: cannot import name 'is_path' from 'PIL._util' (/usr/local/lib/python3.8/dist-packages/PIL/_util.py)
DEBUG:PIL.Image:Importing BmpImagePlugin
DEBUG:PIL.Image:Image: failed to import BmpImagePlugin: cannot import name 'is_path' from 'PIL._util' (/usr/local/lib/python3.8/dist-packages/PIL/_util.py)
DEBUG:PIL.Image:Importing BufrStubImagePlugin
DEBUG:PIL.Image:Image: failed to import BufrStubImagePlugin: cannot import name 'is_path' from 'PIL._util' (/usr/local/lib/python3.8/dist-packages/PIL/_util.py)
DEBUG:PIL.Image:Importing CurImagePlugin
DEBUG:PIL.Image:Image: failed to import CurImagePlugin: cannot import name 'is_path' from 'PIL._util' (/usr/local/lib/python3.8/dist-packages/PIL/_util.py)
DEBUG:PIL.Image:Importing DcxImagePlugin
DEBUG:PIL.Image:Image: failed to import DcxImagePlugin: cannot import name 'is_path' from 'PIL._util' (/usr/local/lib/python3.8/dist-packages/PIL/_util.py)
DEBUG

**3. Extracting the text from each page of the PDF**

In [19]:
# This is the treshold set for text conversion, any page having less than thresholdForTextConversion,
# Will be inserted as an image not text.


def get_text_width(document):
    """
    Returns the text width in mm.
    """
    section = document.sections[0]

    sectionWidth = (section.page_width - section.left_margin - section.right_margin)/36000
    
    return  section, sectionWidth

thresholdForTextConversion = 50
lang = 'eng'  #the language for conversion in case of arabic, just change to 'ar'
document = Document()



#iterating over the pages
for page in iterable: 
    if image == False:
      pix = page.get_pixmap(dpi=300) #save the pdf page as an image
      output = "outfile.png"
      pix.save(output)
    else:
      output = file
    # ocr_dict now holds all the OCR info including text and location on the image 
    ocr_dict = pytesseract.image_to_data(output, lang = lang, output_type=Output.DICT)
    text = " ".join(ocr_dict['text'])    
    text = text.strip() #remove any trailing or leading white spaces

    numWordsInPage = len(text.split())
    #if the page has so many text and thus, not an image
    if numWordsInPage >= thresholdForTextConversion:
        textList = text.split("   ")  #divide the string by double spaces into list
        for txt in textList:
            paragraph = document.add_paragraph(txt)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
            
 
    else: #it is likely a picture
        section, sectionWidth = get_text_width(document)
        paragraph = document.add_picture(output, width = Mm(sectionWidth))
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        
    # Adding a page break
    if len(iterable) != 1:      
      document.add_page_break()

**4. Downloading the files.**

In [20]:
#delete excess files in sessions
for filename in os.listdir():
    if filename.endswith(".pdf") or filename.endswith(".docx"): 
         os.remove(filename)

docName = "output"+ time.strftime("%Y%m%d-%H%M%S") + ".docx"
docNamePdf = (docName[:-4] + "pdf")

document.save(docName)
#pypandoc.convert_file(docName, 'pdf', outputfile= docNamePdf, extra_args=['-V', 'geometry:margin=1.5cm'])
print('Downloading results...')


files.download(docName)  #download docx
#files.download(docNamePdf)  #download pdf


  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>